In [1]:
import numpy as np
from MyGenerator import DataGenerator
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
import pickle
import os

Using TensorFlow backend.


In [2]:
with open(os.path.join("regression_data", 'labels.pickle'), 'rb') as handle:
    labels = pickle.load(handle)

In [3]:
list_of_labels = [*labels]
np.random.shuffle(list_of_labels)

In [4]:
len(labels)

50000

In [5]:
partition = {'train': list_of_labels[:40000], 
             'validation': list_of_labels[40000:45000],
             'test': list_of_labels[45000:50000]}
y_label = {'train': [labels[k] for k in partition['train']], 
           'validation': [labels[k] for k in partition['validation']],
           'test': [labels[k] for k in partition['test']]}

In [6]:
len(y_label['validation'])

5000

In [7]:
len(partition['validation'])

5000

In [8]:
params = {'dim': (2500,),
          'batch_size': 32,
          'n_channels': 1,
          'shuffle': True}
test_params = {'dim': (2500,),
               'batch_size': len(partition['test']),
               'n_channels': 1,
               'shuffle': False}

In [9]:
training_generator = DataGenerator(partition['train'], y_label['train'],  **params)
validation_generator = DataGenerator(partition['validation'], y_label['validation'],  **params)
testing_generator = DataGenerator(partition['test'], y_label['test'],  **test_params)

In [10]:
inputs = keras.Input(shape = (2500,1))
conv_1_1 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(inputs)
conv_1_2 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_1)
conv_1_3 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_2)
pool_1 = keras.layers.MaxPool1D(pool_size = 4)(conv_1_3)

conv_2_1 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_1)
conv_2_2 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_2_1)
#conv_2_3 = keras.layers.Conv1D(64, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_2_2)
pool_2 = keras.layers.MaxPool1D(pool_size = 4)(conv_2_2)

conv_3_1 = keras.layers.Conv1D(128, kernel_size = 7, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_2)
#conv_3_2 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_1)
#conv_3_3 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_2)
pool_3 = keras.layers.MaxPool1D(pool_size = 4)(conv_3_1)



flat_1 = keras.layers.Flatten()(pool_3)



dense_1 = keras.layers.Dense(1024, activation = 'relu', 
                             kernel_initializer = keras.initializers.he_normal())(flat_1)
outputs = keras.layers.Dense(3, kernel_initializer = keras.initializers.he_normal())(dense_1)
model = keras.Model(inputs = inputs, outputs = outputs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2500, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2498, 32)          128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2496, 32)          3104      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2494, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 623, 32)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 619, 64)           10304     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 615, 64)           20544     
__________

In [11]:
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.second_reg.hdf5', 
                               verbose=2, save_best_only=True)
model.compile(optimizer=adam,
              loss='mean_squared_error', metrics = ['mae'])
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              #steps_per_epoch=training_generator.__len__(),
                              #workers=16,
                              callbacks=[checkpointer],
                              epochs = 100,
                              use_multiprocessing=True)

Epoch 1/100
1250/1250 [==============================] - 23s 18ms/step - loss: 17.3457 - mean_absolute_error: 0.4698 - val_loss: 0.0879 - val_mean_absolute_error: 0.1915

Epoch 00001: val_loss improved from inf to 0.08793, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 2/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.0702 - mean_absolute_error: 0.1638 - val_loss: 0.0496 - val_mean_absolute_error: 0.1452

Epoch 00002: val_loss improved from 0.08793 to 0.04963, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 3/100
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0503 - mean_absolute_error: 0.1370 - val_loss: 0.0388 - val_mean_absolute_error: 0.1299

Epoch 00003: val_loss improved from 0.04963 to 0.03881, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 4/100
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0402 - mean_absolute_error: 0.1211 - val_loss: 0.0332 - val_mean_absolute_er

1250/1250 [==============================] - 18s 14ms/step - loss: 0.0102 - mean_absolute_error: 0.0575 - val_loss: 0.0087 - val_mean_absolute_error: 0.0631

Epoch 00032: val_loss did not improve from 0.00794
Epoch 33/100
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0100 - mean_absolute_error: 0.0569 - val_loss: 0.0072 - val_mean_absolute_error: 0.0567

Epoch 00033: val_loss improved from 0.00794 to 0.00721, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 34/100
1250/1250 [==============================] - 18s 15ms/step - loss: 0.0098 - mean_absolute_error: 0.0567 - val_loss: 0.0072 - val_mean_absolute_error: 0.0571

Epoch 00034: val_loss improved from 0.00721 to 0.00720, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 35/100
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0097 - mean_absolute_error: 0.0561 - val_loss: 0.0071 - val_mean_absolute_error: 0.0563

Epoch 00035: val_loss improved from 0.00720 to 0.00709


Epoch 00064: val_loss did not improve from 0.00547
Epoch 65/100
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0070 - mean_absolute_error: 0.0478 - val_loss: 0.0053 - val_mean_absolute_error: 0.0489

Epoch 00065: val_loss improved from 0.00547 to 0.00529, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 66/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.0070 - mean_absolute_error: 0.0478 - val_loss: 0.0053 - val_mean_absolute_error: 0.0490

Epoch 00066: val_loss did not improve from 0.00529
Epoch 67/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.0069 - mean_absolute_error: 0.0474 - val_loss: 0.0052 - val_mean_absolute_error: 0.0485

Epoch 00067: val_loss improved from 0.00529 to 0.00518, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 68/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.0069 - mean_absolute_error: 0.0475 - val_loss: 0.0054 - val_mean_absolute_erro

1250/1250 [==============================] - 17s 14ms/step - loss: 0.0059 - mean_absolute_error: 0.0439 - val_loss: 0.0046 - val_mean_absolute_error: 0.0461

Epoch 00099: val_loss did not improve from 0.00459
Epoch 100/100
1250/1250 [==============================] - 18s 15ms/step - loss: 0.0058 - mean_absolute_error: 0.0438 - val_loss: 0.0045 - val_mean_absolute_error: 0.0455

Epoch 00100: val_loss improved from 0.00459 to 0.00450, saving model to saved_models/weights.best.roof_reg.hdf5


In [12]:
model.load_weights('saved_models/weights.best.second_reg.hdf5')
model.evaluate_generator(testing_generator)

[0.007711098529398441, 0.04658902436494827]

In [13]:
k = 214
X_test = np.zeros((1,2500,1))
X_test[0,] = np.load("regression_data/" + partition['test'][k] + '.npy')[:,np.newaxis]

In [14]:
print(model.predict(X_test), y_label['test'][k] )

[[0.36433044 8.853346   1.5317055 ]] [0.3739514303404323, 8.8847693326925, 1.5479680086560492]


In [15]:
model.save('saved_models/second_model.h5')